## Update ArcGIS Online Feature Layer from Google Sheets


#### Connect to your GIS & import the necessary libraries:

In [ ]:
try:
    import gspread, oauth2client
except ModuleNotFoundError:
    %pip install --quiet gspread oauth2client

In [ ]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from arcgis.gis import GIS
gis = GIS("home")
import arcgis
from arcgis.features import FeatureLayerCollection

#### Import service-account JSON & authorize:

##### Open "Files" above and navigate to your home folder. Upload your json to files. If google sheet is public, skip this step.

If you need to create a json file, follow these steps:
1. Sign in to Google Cloud Console
    - Go to https://console.cloud.google.com/ and log in with the Google account that owns your sheet project.

2. Select or Create a Project
    - At the top, click the project dropdown and either pick the project you’ve been using (the one whose number showed up in your API errors) or click New Project to make a fresh one.

3. Enable the Sheets API (if you haven’t already)
    - In the left menu, go to APIs & Services → Library, search for “Google Sheets API,” click it and hit Enable.

4. Open the Service Accounts Page
    - In the left menu, go to IAM & Admin → Service Accounts.

5. Create a New Service Account
    - Click + CREATE SERVICE ACCOUNT at the top.
    - Give it a name like “sheet-reader” and an optional description.
    - Click Create.

6. Grant It Permissions
    - In the “Grant this service account access to project” step, pick Viewer (or more restrictive roles if you like) under “Project.”
    - If you want to be extra precise, you can grant Sheets Viewer and Drive Viewer roles instead of Project Viewer.
    - Click Continue.

7. Skip Granting Users Access
    - You can just click Done on the next screen (no additional users need to manage this account).

8. Create a JSON Key
    - Back on the Service Accounts list, find your new “sheet-reader” account and click its ⋮ (more actions) menu at the right, then Manage keys.
    - Under the Keys tab, click Add Key → Create new key.
    - Choose JSON, then Create.
    - A file named something like sheet-reader-12345-abcd.json will download automatically.

9. Upload that JSON to your Notebook
    - In your ArcGIS Notebook’s Files pane, navigate into the home folder and click Upload, then pick that JSON file.
    - You’ll now be able to reference it by filename in your code.

10. Share Your Sheet with the Service Account
    - Open your Google Sheet, click Share, and grant Viewer access to the service-account’s email (you’ll find it in the JSON under client_email).

In [ ]:
SCOPE = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

# ── In notebooks, upload your json to files. If google sheet is public, skip this step ──
SERVICE_ACCOUNT_FILE = "/arcgis/home/<YOUR JSON NAME.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(
    SERVICE_ACCOUNT_FILE,
    SCOPE
)
gc = gspread.authorize(creds)

#### Connect to your Google Sheet:

In [ ]:
SHEET_ID = "<ADD SHEET ID FROM URL>"
GID      = "<ADD GID, 0 IF ONE SHEET>" 
CSV_URL  = f"https://docs.google.com/spreadsheets/d/ <SHEETID /export?format=csv&gid= <GID>"
sh = gc.open_by_key(SHEET_ID)
print(sh.title) 
df = pd.DataFrame(sh.get_worksheet(0).get_all_records())
print(df.head())

### Step 1: read the sheet

In [ ]:
df = pd.read_csv(CSV_URL)
print(f"Loaded {len(df)} rows from Google Sheet.")

### Step 2: convert to a SpatialDataFrame
#### Change Long and Lat to column headers of X and Y.

In [ ]:
from arcgis.features import GeoAccessor
import arcgis 

sdf = GeoAccessor.from_xy(
    df,     
    "Long",
    "Lat"
)
sdf.spatial.sr = {"wkid": 4326}

print(f"✅ Converted to SpatialDataFrame with {len(sdf)} rows.")


### Step 2: clear current feature layer table and overwrite with new table

In [ ]:
flc = FeatureLayerCollection.fromitem(gis.content.get("9adcddbddc1e4c14bc5a53b6a661a1f4"))

layer = flc.layers[0]
layer.delete_features(where="1=1")       

fs = sdf.spatial.to_featureset()
res = layer.edit_features(adds=fs)           

print(f"✅ Feature Layer Overwritten with {len(fs)} features. Server response:")
print(res)